# Programmieraufgabe I                                              (9 + 8 + 4 + 4 = 25 Punkte)
## _Abgabe in der Woche vom 5. bis 9. November_

In [2]:
import numpy as np 
from numpy import linalg as LA
import matplotlib.pyplot as plt


Die grundlegenden Operationen der linearen Algebra (d.h. Summen, Skalarprodukte, Norm, Matrix-Vektor-Produkt, etc.) sind sind für numpy-arrays bereits implementiert und dürfen hier verwendet werden. 

## Teilaufgabe a)

Implementieren Sie eine Funktion, die als Input ein Matrix $A$ (als numpy-array) erhält und die reduzierte QR-Zerlegung ($\hat{Q}$, $\hat{R}$ jeweils als numpy-arrays) zurückgibt. Als Subroutine empfiehlt es sich, die Anwendung der Householder-Transformation separat zu implementieren.


In [4]:
def sgn(x):
    if x > 0:
        return 1.
    elif x < 0:
        return -1.
    elif x == 0:
        return 0.
    else:
        return x
def pan_norm(x, begin, n):#pan dimensinale 2-norm
    res = 0.
    for i in xrange(begin, n):
        res += x[i]**2
    return np.sqrt(res)
def myqr(B): 
    ## Diese Funktion soll die reduzierte QR-Zerlegung von A berechnen und Q,R (in der Vorlesung $\hat{Q}, \hat{R}$) zurückgeben...
    (m,n) = B.shape;
    A = np.copy(B)# das R
    Q = np.identity(m)
    
    
    for i in xrange(min(n,m)):
        Q[i][i] = 1
    for i in xrange(n):
        tmp = A[i:m, i:n]
        #print(tmp)
        sig = sgn(tmp[0][0])*LA.norm(tmp.T[0])
        #print(sig)
        w = np.copy(tmp.T[0]).reshape((m-i,1))
        w[0][0] += sig
        w /= LA.norm(w)
        #print(w)
        tmp = householder(tmp, w)
        
        #beim kopieren wird benutz das A 0-Block hat ist
        for ix in xrange(i,m):#ich weiß es gibt iteratoren
            for iy in xrange(i,n):
                A[ix][iy] = tmp[ix-i][iy-i]
        #print(tmp)
        
        tmp2 = Q[i:m, 0:m]
        tmp2 = householder(tmp2, w)
        for ix in xrange(i,m):#ich weiß es gibt iteratoren
            for iy in xrange(0,m):
                Q[ix][iy] = tmp2[ix-i][iy]
        #print(Q)
        #print(np.matmul(Q,B))
        #print(A)
    return Q[0:n,:].T,A[0:n,:]
    #return Q,R

def householder(arg, w): 
    ## Diese Funktion soll eine Householder-Transformation $I - 2ww^T$ auf das Argument arg (Matrix oder Vektor) anwenden.
    (m,n) = arg.shape;
   
    A = np.copy(arg)
    t = np.matmul(w.T, arg)
    t = np.matmul(w, t)
    res = A -2*t
    return res

A = np.array([[1, -2, 4], [1, -1, 1], [1,1,1], [1,2,4]], dtype = float)
A = np.array([[3, 5], [0, 2], [0,0], [4,5]], dtype = float)
q1, r1= myqr(A)
q2, r2 = np.linalg.qr(A)
print(q1-q2)
print(r1-r2)


[[  2.22044605e-16   0.00000000e+00]
 [  0.00000000e+00  -2.22044605e-16]
 [  0.00000000e+00   0.00000000e+00]
 [  1.11022302e-16   1.11022302e-16]]
[[ 0.  0.]
 [ 0.  0.]]


In [46]:
def sgn(x):
    if x > 0:
        return 1.
    elif x < 0:
        return -1.
    elif x == 0:
        return 0.
    else:
        return x
def pan_norm(x, begin, n):#pan dimensinale 2-norm
    res = 0.
    for i in xrange(begin, n):
        res += x[i]**2
    return np.sqrt(res)
def myqr(B): 
    ## Diese Funktion soll die reduzierte QR-Zerlegung von A berechnen und Q,R (in der Vorlesung $\hat{Q}, \hat{R}$) zurückgeben...
    (m,n) = B.shape;
    A = np.copy(B)[:n,:n]# das R
    Q = np.zeros((m,n))
    
    
    for i in xrange(min(n,m)):
        Q[i][i] = 1
    for i in xrange(n):
        tmp = A[i:, i:n]
        #print(tmp)
        sig = sgn(tmp[0][0])*LA.norm(tmp.T[0])
        #print(sig)
        w = np.copy(tmp.T[0]).reshape((n-i,1))
        w[0][0] += sig
        w /= LA.norm(w)
        #print(w)
        tmp = householder(tmp, w)
        
        #beim kopieren wird benutz das A 0-Block hat ist
        for ix in xrange(i,):#ich weiß es gibt iteratoren
            for iy in xrange(i,n):
                A[ix][iy] = tmp[ix-i][iy-i]
        #print(tmp)
        
        tmp2 = Q[i:m, 0:m]
        tmp2 = householder(tmp2, w)
        for ix in xrange(i,2):
            for iy in xrange(0,m):
                Q[ix][iy] = tmp2[ix-i][iy]
        #print(Q)
        #print(np.matmul(Q,B))
        #print(A)
    return Q,A
    #return Q,R

def householder(arg, w): 
    ## Diese Funktion soll eine Householder-Transformation $I - 2ww^T$ auf das Argument arg (Matrix oder Vektor) anwenden.
    (m,n) = arg.shape;
   
    A = np.copy(arg)
    t = np.matmul(w.T, arg)
    t = np.matmul(w, t)
    res = A -2*t
    return res

A = np.array([[1, -2, 4], [1, -1, 1], [1,1,1], [1,2,4]], dtype = float)
A = np.array([[3, 5], [0, 2], [0,0], [4,5]], dtype = float)
q1, r1= myqr(A)
q2, r2 = np.linalg.qr(A)
print(q1[0:2,0:4].T)
print(q2)
print(r1)
print(r2)

ValueError: shapes (1,2) and (4,2) not aligned: 2 (dim 1) != 4 (dim 0)

Testen Sie Ihre eigene Implementierung, indem Sie eine zufällige $5 \times 4$-Matrix erzeugen und deren mit Ihrer Implementierung generierte QR-Zerlegung mit der in numpy bereits implementierten QR-Zerlegung vergleichen.

In [13]:

## Hinweis: Verwenden Sie die Generierung zufälliger Matrizen aus numpy... 
for _ in xrange(100):
    A = np.random.rand(5,4)
    q1, r1= myqr(A)
    q2, r2 = np.linalg.qr(A)
    err = LA.norm(q1-q2)+LA.norm(r1-r2)
    print( "Fehler = "+str(err))
    if(err>10e-6):print("Fehler zu gross")
    


Fehler = 2.60633858017e-15
Fehler = 3.33027921391e-15
Fehler = 1.54885069368e-15
Fehler = 1.90527248225e-15
Fehler = 3.27194301966e-15
Fehler = 3.0511826216e-15
Fehler = 1.22832520235e-15
Fehler = 1.73546607768e-15
Fehler = 2.65120642604e-15
Fehler = 1.35874271552e-15
Fehler = 2.32490063653e-15
Fehler = 1.12017920617e-14
Fehler = 1.19940650354e-15
Fehler = 1.99798035966e-15
Fehler = 1.04449597333e-15
Fehler = 1.48031996919e-15
Fehler = 1.51966440836e-15
Fehler = 1.43718205034e-15
Fehler = 2.16444325034e-15
Fehler = 2.43981265387e-15
Fehler = 1.44729392988e-15
Fehler = 1.61997065554e-15
Fehler = 2.62755414783e-15
Fehler = 1.59015277361e-15
Fehler = 1.36208205164e-15
Fehler = 2.88231866798e-15
Fehler = 3.09054380075e-15
Fehler = 1.65475234087e-15
Fehler = 5.51478528909e-15
Fehler = 2.32671408985e-15
Fehler = 1.2245654742e-15
Fehler = 2.01652036782e-15
Fehler = 1.66072733466e-15
Fehler = 1.85918683822e-15
Fehler = 3.8552848739e-15
Fehler = 2.01480254168e-15
Fehler = 1.59062770956e-15
Fehl

## Teilaufgabe b) 


Implementieren Sie die QR-basierte Lösungsstrategie für das Least-Squares Problem $\min_x \lVert Ax - b \rVert$: Die Funktion soll als Input die Matrix $A$ und den Vektor $b$ (beide als numpy-array) erhalten und die Lösung $x_{\min}$ zurückgegebn. 

Die Lösung eines Gleichungsystems mit oberer Dreiecksstruktur sollen Sie als eigene Routine programmieren. (Der lineare Löser aus numpy soll hier nicht verwendet werden.)

In [4]:
def myuppertriangularsolver(A,b):
    
    ## Löst die Gleichung Ax = b, wobei A eine obere Dreiecksmatrix mit strikt positiven Diagonalementen ist. ##
    
    return x

def myleastsquares(A,b):
    
    ## Löst das least squares problem $min_x \lVert Ax-b \rVert_2$. 
    
    return xmin 

Testen Sie Ihre Implementierung an Aufgabe 1b) von Übungsblatt Nr. 2: 

In [66]:
b = np.array([2, 3, 3, -2])
A = np.array([[1, -2, 4], [1, -1, 1], [1,1,1], [1,2,4]])
xtrue=np.array([4, -4./5, -1])

## Test: ...

## Teilaufgabe c) 

Es sei das Interval $I = (0,1)$ gegeben. Die folgende Funktion gibt einen Vektor von $x-$Werten aus $I$ sowie die geringfügig zufällig gestörten zugehörigen $y-$Werte $y \approx f(x)$ zurück.

In [58]:
def noisyfunction(fun, numberofpoints, noisemax):
    xvals = np.random.rand(numberofpoints)
    yvals = np.zeros(numberofpoints)
    for k in range(numberofpoints):
        yvals[k] = fun(xvals[k]) + 2*noisemax*(np.random.rand()-0.5)
    return xvals, yvals

Es sei eine Funktion $F: \mathbb{R} \times \mathbb{N} \rightarrow \mathbb{R}$ gegeben, z.B. $F_{\text{Pol}}(x,l) := x^l$ oder $F_{\sin}(x,l) = \sin(\pi lx)$, derart, dass durch $f_l(x) := F(x,l)$ eine "sinnvolle" Familie von Basisfunktionen definiert wird. 

Für gegebene Datenpunkte $(x_1, y_1), ..., (x_m, y_m)$ und $n \in \mathbb{N}$ wollen wir einen Koeffizientenvektor $c = (c_0, ..., c_n)$ bestimmen, der das Least-Squares Problem
$\min_{c \in \mathbb{R}^n} \sum_{k=1}^m \left| \sum_{l=1}^n c_l f_l(x_k) - y_k \right|^2 $ löst. Schreiben Sie dazu die Funktion "myregression" mit folgenden Inputs: 
* xvals, yvals: Arrays mit den gegebenen Wertepaaren
* funfamily: Funktion $F$ für die Erzeugung der Basisfunktionen (siehe oben)
* degree: entspricht $n$, der Anzahl der verwendeten Basisfunktionen
* xtoevaluate: array mit $x-$Werten, an denen die gefundene Regressions-Lösung ausgewertet werden soll

Ausgegeben werden soll der Koeffizientenvektor coeffvec sowie die Auswertung y der Regressions-Lösung an den in xtoevaluate gegebenen x-Werten.

In [68]:
def myregression(xvals, yvals, funfamily, degree, xtoevaluate):
   
    ## ... ## 

    return y, coeffvec

## Implementierung der Basis-Familien $F_Pol$ bzw. $F_sin$ von oben: 
def polynomialfamily(x,l):
    return x**l

def sinefamily(x,l):
    return np.sin(np.pi*x*(l+1))

## Teilaufgabe d) 

Testen Sie Ihre Implementierung für die Lösung des Least-Squares Problems mit den unten erzeugten Wertepaaren: 

* Plotten Sie dazu jeweils die verrauschten Wertepaare sowie die Regressionsfunktion.  

* Variieren Sie die Anzahl der Basisfunktionen bzw. die Wahl der Familie von Basisfunktionen.  

Erklären Sie, weshalb die Lösung des Least-Squares Problems für die Approximation von $x-\frac{1}{2}$ mittels Sinus-Schwingungen sehr viel instabiler zu sein scheint als die Approximation des Sinus mit Polynomen.

In [71]:
def fun_1(x):
    return np.sin(2*np.pi*x)
xvals_1,yvals_1=noisyfunction(fun_1, 50, 0.1)

def fun_2(x):
    return x-0.5
xvals_2, yvals_2 = noisyfunction(fun_2, 50, 0.1)

## ...